# Api URLS
## https://www.alphavantage.co/documentation/
# Feel free to add your keys to config.py.  We can import them as needed in the notebook as variables

In [23]:
# import dependencies
import requests
import json
from pandas.io.json import json_normalize # deal with nested columns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Config file keys.  We can import multiple keys variales in here
from config import av_key # api key for alpha vantage

In [21]:
# will need function, symbol, and api key in order to import the json from the api
url_alpha_vantage = "https://www.alphavantage.co/" 

# url variables
function = "TIME_SERIES_DAILY"
symbol = "MSFT"
outputsize = "compact" # full or compact (100 latest vs all data)
query = url_alpha_vantage + "query?" + "function=" + function + "&symbol=" + symbol +"&outputsize=" + outputsize + "&apikey=" + av_key  
url_response = requests.get(query).json()

# pretty print the json
print(json.dumps(url_response, indent=4, sort_keys=True))

{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "MSFT",
        "3. Last Refreshed": "2019-03-01",
        "4. Output Size": "Compact",
        "5. Time Zone": "US/Eastern"
    },
    "Time Series (Daily)": {
        "2018-10-05": {
            "1. open": "112.6300",
            "2. high": "113.1700",
            "3. low": "110.6400",
            "4. close": "112.1300",
            "5. volume": "29068859"
        },
        "2018-10-08": {
            "1. open": "111.6600",
            "2. high": "112.0300",
            "3. low": "109.3400",
            "4. close": "110.8500",
            "5. volume": "29640588"
        },
        "2018-10-09": {
            "1. open": "111.1400",
            "2. high": "113.0800",
            "3. low": "110.8000",
            "4. close": "112.2600",
            "5. volume": "26198594"
        },
        "2018-10-10": {
            "1. open": "111.2400",
            "2. high": "11

# Data frame from dictionary
## df = pd.DataFrame.from_dict(data, orient='columns')


In [30]:
df = pd.DataFrame.from_dict(url_response, orient = 'columns')
df

,Meta Data,Time Series (Daily)
1. Information,"Daily Prices (open, high, low, close) and Volumes",NaN
2. Symbol,MSFT,NaN
2018-10-05,NaN,"{'1. open': '112.6300', '2. high': '113.1700',..."
2018-10-08,NaN,"{'1. open': '111.6600', '2. high': '112.0300',..."
2018-10-09,NaN,"{'1. open': '111.1400', '2. high': '113.0800',..."
2018-10-10,NaN,"{'1. open': '111.2400', '2. high': '111.5000',..."
2018-10-11,NaN,"{'1. open': '105.3500', '2. high': '108.9300',..."
2018-10-12,NaN,"{'1. open': '109.0100', '2. high': '111.2400',..."
2018-10-15,NaN,"{'1. open': '108.9100', '2. high': '109.4800',..."
2018-10-16,NaN,"{'1. open': '109.5400', '2. high': '111.4100',..."
